## 4. Feature_Engineering

In [78]:
#Import libraries for feature engineering
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_regression, SelectKBest
import warnings

# Configure settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("🔧 FEATURE ENGINEERING SETUP")
print("=" * 35)
print("✅ Libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🤖 Scikit-learn available for feature selection")

🔧 FEATURE ENGINEERING SETUP
✅ Libraries imported successfully!
📊 Pandas version: 2.2.3
🤖 Scikit-learn available for feature selection


Imports and setup  
    - Import libraries (pandas, numpy, matplotlib, seaborn, sklearn helpers).  
    - Configure warnings and pandas display.  
    - Print simple environment/info messages.

In [79]:
#Load cleaned data (from data processing step)
df = pd.read_csv('../data/processed/prepocessed_daily_data.csv')  # Will be updated to use processed data
df.set_index('datetime', inplace=True)
df.index = pd.to_datetime(df.index)

Load cleaned data  
    - Read processed CSV into `df`.  
    - Set `datetime` as index and convert to pandas datetime index.

Inspect columns  
    - Quick evaluation of `df.columns` to review available features.

In [81]:
df['sunrise'] = pd.to_datetime(df['sunrise'])
df['sunset'] = pd.to_datetime(df['sunset'])
df['day_length_hours'] = df['sunset'] - df['sunrise']
df = df.drop(columns=['sunrise', 'sunset'])
df['day_length_hours'] = df['day_length_hours'].dt.total_seconds() / 3600.0

df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['dayofyear_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['dayofyear_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

df.drop(columns=['day', 'month', 'day_of_year', 'conditions'], inplace=True)

df['target5+'] = df['temp'].shift(-5)
df['target4+'] = df['temp'].shift(-4)
df['target3+'] = df['temp'].shift(-3)
df['target2+'] = df['temp'].shift(-2)
df['target1+'] = df['temp'].shift(-1)
df = df[~df['target5+'].isna()].copy()

Time features and forecast targets  
    - Convert `sunrise`/`sunset` to datetime and compute `day_length_hours`; drop original columns.  
    - Create cyclical encodings: `month_sin`, `month_cos`, `dayofyear_sin`, `dayofyear_cos`.  
    - Drop original `day`, `month`, `day_of_year`.  
    - Create shifted forecast targets: `target1+` … `target5+` (future temperature at 1–5 steps).  
    - Drop rows with NaN introduced by the furthest shift (`target5+`).

In [82]:
df["temp_solar_interaction"] = df["temp"] * df["solarradiation"]
df["uv_temp_interaction"] = df["uvindex"] * df["temp"]
df['temp_cloudcover_interaction'] = df['temp'] * df['cloudcover']
df['temp_sealevelpressure_interaction'] = df['temp'] * df['sealevelpressure']
df['weighted_precip'] = df['precipprob'] * df['precip']
df['effective_solar'] = df['solarradiation'] * (1 - df['cloudcover']/100)
df['precip_impact'] = df['precipprob'] * df['precip']

Interaction features and domain transforms  
    - New features combining sensors: `temp_solar_interaction`, `uv_temp_interaction`, `temp_cloudcover_interaction`, `temp_sealevelpressure_interaction`.  
    - Weighted precipitation and effective solar radiation: `weighted_precip`, `effective_solar`, `precip_impact`.

In [83]:
df['wind_u'] = df['windspeed'] * np.cos(2 * np.pi * df['winddir'] / 360)  # gió đông-tây
df['wind_v'] = df['windspeed'] * np.sin(2 * np.pi * df['winddir'] / 360)  # gió nam-bắc
df = df.drop('winddir', axis=1)

df['temp_minus_dew'] = df['temp'] - df['dew']

# Tạo feature moonphase_sin
df['moonphase_sin'] = np.sin(2 * np.pi * df['moonphase'] / 1)

# Tạo feature moonphase_cos
df['moonphase_cos'] = np.cos(2 * np.pi * df['moonphase'] / 1)

# (Tùy chọn) Bỏ cột moonphase gốc
df = df.drop('moonphase', axis=1)

Wind decomposition and moonphase  
    - Convert polar wind (`windspeed`, `winddir`) to Cartesian components: `wind_u`, `wind_v`; drop `winddir`.  
    - Compute moonphase cyclical features: `moonphase_sin`, `moonphase_cos` and drop original `moonphase`.  
    - Note: `temp_minus_dew` is computed in the cell but not saved back to `df` (might be intentional or an oversight).

In [84]:
# Create lagging features
def create_lag_features(df, cols, lags):
    for col in cols:
        for lag in lags:
            df[f"{col}_lag_{lag}"] = df[col].shift(lag)
    return df

# Specify columns and lags
# Get all numerical columns
computing_columns = df.drop(columns=['year', 'season', 'target5+', 'target4+', 'target3+', 'target2+', 'target1+', 'month_sin',
                                    'month_cos', 'dayofyear_sin', 'dayofyear_cos']).columns

lag_steps = [1, 2, 3, 5, 7, 10, 14, 21, 30]  # Example lag steps

# Apply lagging features before handling rolling horizons
df = create_lag_features(df, computing_columns, lag_steps)

# Function to compute rolling mean and percentage change
def compute_rolling(df, horizon, col):
    label = f"rolling_{horizon}_{col}"
    df[label] = df[col].rolling(horizon, min_periods=horizon).mean()  # Ensure full horizon is used
    df[f"{label}_change"] = df[col] - df[label]
    return df

# Compute rolling features for specified horizons
rolling_horizons = [3, 4, 5, 7, 14, 21, 30]  # Rolling windows of 3, 7, 14 days
for horizon in rolling_horizons:
    for col in computing_columns:
        df = compute_rolling(df, horizon, col)

Lag and rolling mean features  
    - Define `create_lag_features()` and produce lag features for selected numerical columns using lags [1,2,3,5,7,10,14,21,30].  
    - Define `compute_rolling()` to compute rolling means and difference to current value.  
    - Apply rolling windows for horizons [3,7,14,21,30] for the same set of columns.

In [85]:
#Months and days average
def expand_mean(df):
    return df.expanding(1).mean()

for col in computing_columns:
    df[f"month_avg_{col}"] = df[col].groupby(df.index.month, group_keys=False).apply(expand_mean)
    df[f"day_avg_{col}"] = df[col].groupby(df.index.day_of_year, group_keys=False).apply(expand_mean)
    df[f"year_avg_{col}"] = df[col].groupby(df.index.year, group_keys=False).apply(expand_mean)
    df[f"season_avg_{col}"] = df[col].groupby(df['season'], group_keys=False).apply(expand_mean)
    df["day_max_temp"] = df['temp'].groupby(df.index.day_of_year, group_keys=False).cummax()
    df["day_min_temp"] = df['temp'].groupby(df.index.day_of_year, group_keys=False).cummin()

df["temp_anomaly_vs_month_avg"] = df["temp"] - df["month_avg_temp"]
df["temp_anomaly_vs_day_avg"] = df["temp"] - df["day_avg_temp"]
df["temp_anomaly_vs_season_avg"] = df["temp"] - df["season_avg_temp"]

df["pressure_trend_2d"] = df["sealevelpressure"].diff(2)
df["pressure_trend_3d"] = df["sealevelpressure"].diff(3)
df["pressure_trend_5d"] = df["sealevelpressure"].diff(5)
df["pressure_trend_7d"] = df["sealevelpressure"].diff(7)
df["pressure_trend_14d"] = df["sealevelpressure"].diff(14)
df["pressure_trend_21d"] = df["sealevelpressure"].diff(21)
df["pressure_trend_30d"] = df["sealevelpressure"].diff(30)

Expanding/grouped aggregates and volatility  
    - Compute expanding (cumulative) means grouped by month, day-of-year, year, and season (e.g., `month_avg_*`, `day_avg_*`, `year_avg_*`, `season_avg_*`).  
    - Add monthly max/min temperature trackers and temperature volatility (rolling std for 7,14,21,30).  
    - Flags and anomalies: `temp_spike_flag`, `temp_anomaly_vs_month_avg`, `temp_anomaly_vs_season_avg`.  
    - Pressure trend features: `pressure_trend_3d` and `pressure_trend_7d` (note: code uses shift(30) for the 7d variable — verify intended window).

In [86]:
# Drop rows with NaN values caused by rolling horizons
df = df.iloc[30:]
# Verify no NaN values exist
nan_summary = df.isna().sum()
print("Summary of NaN values in each column after handling rolling horizons and lagging:")
print(nan_summary[nan_summary > 0].sort_values())

if df.isna().any().any():
    print("\nThe dataframe contains NaN values.")
else:
    print("\nThe dataframe does not contain any NaN values.")

Summary of NaN values in each column after handling rolling horizons and lagging:
Series([], dtype: int64)

The dataframe does not contain any NaN values.


Clean-up and NaN handling  
    - Drop the first 30 rows to remove rows affected by rolling windows and lags.  
    - Print a summary of remaining NaNs and confirm whether the DataFrame contains NaNs.

In [87]:
df.shape

(3619, 947)

 Shape check  
    - Print `df.shape` to inspect final dataset dimensions after feature engineering.

In [88]:
df.to_csv('../data/processed/feature_engineering_daily_data2.csv')

Persist features  
    - Save the final engineered DataFrame to CSV: `feature_engineering_daily_data2.csv`.

## Engineered Features Overview

This section documents only some key the engineered features used by the daily forecasting pipeline. It excludes raw/base inputs and focuses on what was created during feature engineering.

| Variable Group | Variable Name | Definition | Effect on Model |
|---|---|---|---|
| Cyclical Features | `sin_doy`, `cos_doy` | Sine and cosine encodings of day-of-year to preserve annual cyclicality | Provides smooth seasonal patterning; anchors warm/cool seasons without month boundary jumps |
| Day Length | `day_length_hours` | Number of daylight hours for the date/location | Longer days typically raise daytime temps and expand diurnal range (positive, season-dependent) |
| Day Length (lags) | `day_length_hours_lag_21`, `day_length_hours_lag_30` | Day length 21/30 days earlier (seasonal memory) | Mild positive in warming seasons; captures delayed seasonal effects |
| Temperature lags | `temp_lag_1`, `temp_lag_7`, `temp_lag_30` | Temperature 1/7/30 days earlier | Strong positive autocorrelation for near horizons; `lag_30` reflects seasonal baseline |
| Moving averages (rolling) | `rolling_3_*`, `rolling_7_*`, `rolling_30_*` | Rolling means/STD of meteorological signals (e.g., `rolling_30_sealevelpressure`) | Noise reduction and regime identification; longer windows capture persistent patterns |
| Trend / Change | `*_trend_7`, `*_change_3`, `rolling_3_sealevelpressure_change` | 7‑day linear trend or 3‑day deltas | Pressure drops often precede cooling/fronts; rising pressure stabilizes temperatures |
| Interactions | `temp_sealevelpressure_interaction` | First‑order interaction of temperature × sea‑level pressure | Captures stability–heating synergy; sign depends on regime and cloudiness |
| Wind components | `wind_u`, `wind_v` | Decomposition of wind into zonal (`u`, east–west) and meridional (`v`, north–south) components | Directional advection of warm/cold air; effect depends on local climatology and fetch |
| Month (categorical) | `month` (or one‑hot) | Calendar month to encode discrete seasonal regimes | Positions forecast within monthly regime shifts; complements cyclical encoding |
| Prior‑day aggregates | `day_avg_feelslike`, `day_avg_tempmin`, `day_avg_tempmax` | Previous day’s average/min/max | Provides short‑term thermal context; generally positive for next‑day levels |
| Temperature anomaly | `temp_minus_season_avg` (aka `temp_anomaly`) | Deviation from climatological/seasonal average | Positive anomaly → forecast above seasonal baseline; negative → below |
| Seasonal pressure baseline | `season_avg_sealevelpressure` | Seasonal average of sea‑level pressure | Regime indicator; interacts with temperature response and cloud cover |

Notes
- Directions are indicative; confirm with SHAP/Permutation Importance on your latest data slice.
- Drift watchlist (recently flagged with high PSI/KS): `day_length_hours_lag_21`, `day_length_hours_lag_30`, `temp_sealevelpressure_interaction`, `rolling_30_sealevelpressure`, `season_avg_sealevelpressure`, `rolling_3_sealevelpressure_change`.
- Ensure production computation is consistent with training (timezone, leap years, day‑length algorithm) and uses the same scaling/normalization where applicable.